In [ ]:
import re
import numpy as np
from collections import Counter
from scipy.spatial.distance import cosine
from scipy import linalg
import matplotlib.pyplot as plt
%matplotlib inline

#Сравнение предложений
Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:
* кошки (животные)
* UNIX-утилита cat для вывода содержимого файлов
* версии операционной системы OS X, названные в честь семейства кошачьих

Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().

In [ ]:
path = 'cat_sentences.txt'
with open(path,'r') as f:
  lines = [k.lower().rstrip() for k in f.readlines()]

In [ ]:
lines

['in comparison to dogs, cats have not undergone major changes during the domestication process.',
 'as cat simply catenates streams of bytes, it can be also used to concatenate binary files, where it will just concatenate sequence of bytes.',
 'a common interactive use of cat for a single file is to output the content of a file to standard output.',
 'cats can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small animals.',
 'in one, people deliberately tamed cats in a process of artificial selection, as they were useful predators of vermin.',
 'the domesticated cat and its closest wild ancestor are both diploid organisms that possess 38 chromosomes and roughly 20,000 genes.',
 'domestic cats are similar in size to the other members of the genus felis, typically weighing between 4 and 5 kg (8.8 and 11.0 lb).',
 'however, if the output is piped or redirected, cat is unnecessary.',
 'cat with one named file is safer where human error i

Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t).

In [ ]:
tokens = []
for line in lines:
  tokens.append(re.split('[^a-z]', line))

In [ ]:
for tok in tokens:
  print(tok)

['in', 'comparison', 'to', 'dogs', '', 'cats', 'have', 'not', 'undergone', 'major', 'changes', 'during', 'the', 'domestication', 'process', '']
['as', 'cat', 'simply', 'catenates', 'streams', 'of', 'bytes', '', 'it', 'can', 'be', 'also', 'used', 'to', 'concatenate', 'binary', 'files', '', 'where', 'it', 'will', 'just', 'concatenate', 'sequence', 'of', 'bytes', '']
['a', 'common', 'interactive', 'use', 'of', 'cat', 'for', 'a', 'single', 'file', 'is', 'to', 'output', 'the', 'content', 'of', 'a', 'file', 'to', 'standard', 'output', '']
['cats', 'can', 'hear', 'sounds', 'too', 'faint', 'or', 'too', 'high', 'in', 'frequency', 'for', 'human', 'ears', '', 'such', 'as', 'those', 'made', 'by', 'mice', 'and', 'other', 'small', 'animals', '']
['in', 'one', '', 'people', 'deliberately', 'tamed', 'cats', 'in', 'a', 'process', 'of', 'artificial', 'selection', '', 'as', 'they', 'were', 'useful', 'predators', 'of', 'vermin', '']
['the', 'domesticated', 'cat', 'and', 'its', 'closest', 'wild', 'ancestor

Не забудьте удалить пустые слова после разделения. Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.

In [ ]:
new_tokens = []
dict_word = {}
i = 0
for token in tokens:
  line = []
  for tok in token:
    if tok != '':
      line.append(tok)
      if tok not in dict_word.keys():
        dict_word.update({tok:i})
        i += 1
  new_tokens.append(line)

In [ ]:
for tok in new_tokens:
  print(tok)

['in', 'comparison', 'to', 'dogs', 'cats', 'have', 'not', 'undergone', 'major', 'changes', 'during', 'the', 'domestication', 'process']
['as', 'cat', 'simply', 'catenates', 'streams', 'of', 'bytes', 'it', 'can', 'be', 'also', 'used', 'to', 'concatenate', 'binary', 'files', 'where', 'it', 'will', 'just', 'concatenate', 'sequence', 'of', 'bytes']
['a', 'common', 'interactive', 'use', 'of', 'cat', 'for', 'a', 'single', 'file', 'is', 'to', 'output', 'the', 'content', 'of', 'a', 'file', 'to', 'standard', 'output']
['cats', 'can', 'hear', 'sounds', 'too', 'faint', 'or', 'too', 'high', 'in', 'frequency', 'for', 'human', 'ears', 'such', 'as', 'those', 'made', 'by', 'mice', 'and', 'other', 'small', 'animals']
['in', 'one', 'people', 'deliberately', 'tamed', 'cats', 'in', 'a', 'process', 'of', 'artificial', 'selection', 'as', 'they', 'were', 'useful', 'predators', 'of', 'vermin']
['the', 'domesticated', 'cat', 'and', 'its', 'closest', 'wild', 'ancestor', 'are', 'both', 'diploid', 'organisms', 't

In [ ]:
print(dict_word)

{'in': 0, 'comparison': 1, 'to': 2, 'dogs': 3, 'cats': 4, 'have': 5, 'not': 6, 'undergone': 7, 'major': 8, 'changes': 9, 'during': 10, 'the': 11, 'domestication': 12, 'process': 13, 'as': 14, 'cat': 15, 'simply': 16, 'catenates': 17, 'streams': 18, 'of': 19, 'bytes': 20, 'it': 21, 'can': 22, 'be': 23, 'also': 24, 'used': 25, 'concatenate': 26, 'binary': 27, 'files': 28, 'where': 29, 'will': 30, 'just': 31, 'sequence': 32, 'a': 33, 'common': 34, 'interactive': 35, 'use': 36, 'for': 37, 'single': 38, 'file': 39, 'is': 40, 'output': 41, 'content': 42, 'standard': 43, 'hear': 44, 'sounds': 45, 'too': 46, 'faint': 47, 'or': 48, 'high': 49, 'frequency': 50, 'human': 51, 'ears': 52, 'such': 53, 'those': 54, 'made': 55, 'by': 56, 'mice': 57, 'and': 58, 'other': 59, 'small': 60, 'animals': 61, 'one': 62, 'people': 63, 'deliberately': 64, 'tamed': 65, 'artificial': 66, 'selection': 67, 'they': 68, 'were': 69, 'useful': 70, 'predators': 71, 'vermin': 72, 'domesticated': 73, 'its': 74, 'closest': 

Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение.

In [ ]:
matrix = np.zeros((len(lines), len(dict_word)))
matrix.shape

(22, 254)

In [ ]:
count = []
for token in new_tokens:
  count.append(dict(Counter(token)))

In [ ]:
for co in count:
  print(count)

[{'in': 1, 'comparison': 1, 'to': 1, 'dogs': 1, 'cats': 1, 'have': 1, 'not': 1, 'undergone': 1, 'major': 1, 'changes': 1, 'during': 1, 'the': 1, 'domestication': 1, 'process': 1}, {'as': 1, 'cat': 1, 'simply': 1, 'catenates': 1, 'streams': 1, 'of': 2, 'bytes': 2, 'it': 2, 'can': 1, 'be': 1, 'also': 1, 'used': 1, 'to': 1, 'concatenate': 2, 'binary': 1, 'files': 1, 'where': 1, 'will': 1, 'just': 1, 'sequence': 1}, {'a': 3, 'common': 1, 'interactive': 1, 'use': 1, 'of': 2, 'cat': 1, 'for': 1, 'single': 1, 'file': 2, 'is': 1, 'to': 2, 'output': 2, 'the': 1, 'content': 1, 'standard': 1}, {'cats': 1, 'can': 1, 'hear': 1, 'sounds': 1, 'too': 2, 'faint': 1, 'or': 1, 'high': 1, 'in': 1, 'frequency': 1, 'for': 1, 'human': 1, 'ears': 1, 'such': 1, 'as': 1, 'those': 1, 'made': 1, 'by': 1, 'mice': 1, 'and': 1, 'other': 1, 'small': 1, 'animals': 1}, {'in': 2, 'one': 1, 'people': 1, 'deliberately': 1, 'tamed': 1, 'cats': 1, 'a': 1, 'process': 1, 'of': 2, 'artificial': 1, 'selection': 1, 'as': 1, 'the

In [ ]:
i = 0
for co in count:
  for k, v in co.items():
    matrix[i][dict_word[k]] = v
  i+=1

In [ ]:
print(matrix)

[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 2. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 1. 1.]]


Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. 

In [ ]:
cos_dist = []
for k in range(matrix.shape[0]):
  cos_dist.append([k, cosine(matrix[0], matrix[k])])

In [ ]:
cos_dist.sort(key=lambda x: x[1])
pprint(cos_dist)

[[0, 0.0],
 [6, 0.7327387580875756],
 [4, 0.7770887149698589],
 [21, 0.8250364469440588],
 [10, 0.8328165362273942],
 [12, 0.8396432548525454],
 [16, 0.8406361854220809],
 [20, 0.8427572744917122],
 [2, 0.8644738145642124],
 [13, 0.8703592552895671],
 [14, 0.8740118423302576],
 [11, 0.8804771390665607],
 [8, 0.8842724875284311],
 [19, 0.8885443574849294],
 [3, 0.8951715163278082],
 [9, 0.9055088817476932],
 [7, 0.9258750683338899],
 [5, 0.9402385695332803],
 [15, 0.9442721787424647],
 [18, 0.9442721787424647],
 [1, 0.9527544408738466],
 [17, 0.956644501523794]]


Совпадают ли ближайшие два предложения по тематике с первым? Совпадают ли тематики у следующих по близости предложений?

In [ ]:
print(new_tokens[0])
print(new_tokens[4])
print(new_tokens[6])

['in', 'comparison', 'to', 'dogs', 'cats', 'have', 'not', 'undergone', 'major', 'changes', 'during', 'the', 'domestication', 'process']
['in', 'one', 'people', 'deliberately', 'tamed', 'cats', 'in', 'a', 'process', 'of', 'artificial', 'selection', 'as', 'they', 'were', 'useful', 'predators', 'of', 'vermin']
['domestic', 'cats', 'are', 'similar', 'in', 'size', 'to', 'the', 'other', 'members', 'of', 'the', 'genus', 'felis', 'typically', 'weighing', 'between', 'and', 'kg', 'and', 'lb']


Разумеется, использованный вами метод крайне простой. Например, он не учитывает формы слов (так, cat и cats он считает разными словами, хотя по сути они означают одно и то же), не удаляет из текстов артикли и прочие ненужные слова.